In [2]:
import os
import os.path as osp
import numpy as np
from PIL import Image
from tqdm import tqdm

In [3]:
def load_depth(depth_path):
    im = Image.open(depth_path)
    im_arr = np.array(im).astype(float)/256
    return im_arr

In [5]:
DATA_DIR = "/scratch1/ganesang/kitti/datasets"

split_file = "kitti_eigen_train.txt"

with open(osp.join(DATA_DIR, split_file), "r") as f:
    data = [line for line in f]

clean_count = 0
half_occ_count = 0
min_depth = 0.01

new_split_data = []
for line in tqdm(data):
    path = line.strip().split(" ")[1]
    semidense_path = osp.join(DATA_DIR, path)
    raw_depth_path = semidense_path.replace("groundtruth", "groundtruth_raw")
    clean_depth_path = semidense_path.replace("groundtruth", "groundtruth_filter")
    half_occ_depth_path = semidense_path.replace("groundtruth", "groundtruth_half_occ")

    semidense_depth = load_depth(semidense_path)
    raw_depth = load_depth(raw_depth_path)
    clean_depth = load_depth(clean_depth_path)
    half_occ_depth = load_depth(half_occ_depth_path)
    
    semidense_mask = semidense_depth > min_depth
    raw_mask = raw_depth > min_depth
    clean_mask = clean_depth > min_depth
    half_occ_mask = half_occ_depth > min_depth

    pts_rm_clean = raw_mask & ~clean_mask
    pts_rm_half_occ = raw_mask & ~half_occ_mask
    
    avail_pts_rm_clean = np.sum(pts_rm_clean & semidense_mask)
    avail_pts_rm_half_occ = np.sum(pts_rm_half_occ & semidense_mask)

    # if avail_pts_rm_clean == 0:
    #     clean_count += 1
    # if avail_pts_rm_half_occ == 0:
    #     half_occ_count += 1
    if avail_pts_rm_clean  > 0 and avail_pts_rm_half_occ > 0:
        new_split_data.append(line)

with open(osp.join("splits", "kitti_eigen_half_occ_train.txt"), "w") as f:
    f.writelines(new_split_data)

100%|██████████| 23158/23158 [12:46<00:00, 30.22it/s]  
